Using any data set you want, implement colabs for the following ANN algorithms (see hints)

Write proper documentation in read.me file and colabs 

a) LSH

b) exhaustive search

c) product quantization

d) trees and graphs

e) hnsw

I am using the Netflix titles dataset available at https://www.kaggle.com/shivamb/netflix-shows

Load the Google Drive to access the dataset

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Install the required libraries

In [3]:
!python -m pip install --upgrade faiss faiss-gpu
!pip install datasketch
!pip install sentence_transformers
!pip install hnswlib
!pip install nmslib
!pip install annoy

Import the dependencies

In [4]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sn
import math
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
import nmslib
import hnswlib

Load the dataset and check the shape

In [5]:
df = pd.read_csv('/content/gdrive/MyDrive/msse/fall-21/data-mining/netflix_titles.csv')
df.shape

(8807, 12)

In [6]:
df.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [7]:
df.drop(columns=["country", "date_added", "type", "rating", "duration", "cast", "show_id"], inplace= True)
df["text"] = df["title"] + " " + df["description"]
df.head(10)

,title,director,release_year,listed_in,description,text
0,Dick Johnson Is Dead,Kirsten Johnson,2020,Documentaries,"As her father nears the end of his life, filmm...",Dick Johnson Is Dead As her father nears the e...
1,Blood & Water,NaN,2021,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...","Blood & Water After crossing paths at a party,..."
2,Ganglands,Julien Leclercq,2021,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Ganglands To protect his family from a powerfu...
3,Jailbirds New Orleans,NaN,2021,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...","Jailbirds New Orleans Feuds, flirtations and t..."
4,Kota Factory,NaN,2021,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Kota Factory In a city of coaching centers kno...
5,Midnight Mass,Mike Flanagan,2021,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,Midnight Mass The arrival of a charismatic you...
6,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha",2021,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,My Little Pony: A New Generation Equestria's d...
7,Sankofa,Haile Gerima,1993,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...","Sankofa On a photo shoot in Ghana, an American..."
8,The Great British Baking Show,Andy Devonshire,2021,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,The Great British Baking Show A talented batch...
9,The Starling,Theodore Melfi,2021,"Comedies, Dramas",A woman adjusting to life after a loss contend...,The Starling A woman adjusting to life after a...


To effeciently run on colab, just use the first 1000 records. As using more data to process caused the colab to crash as it is using high-RAM.

In [8]:
df = df[0:1000]
df.shape

(1000, 6)

**Locality Sensitive Hashing**

In [9]:
import re
import time
from datasketch import MinHash, MinHashLSHForest

Create a function to preprocess the data:

In [10]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [11]:
# Function to get the forest

def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [12]:
# Function for recommending results based on similar query terms
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [13]:
permutations = 128

num_recommendations = 1

Create the forest and search it based on a search query and get the recommended titles.

In [14]:
forest = get_forest(df, permutations)

It took 1.8603973388671875 seconds to build forest.


In [15]:
num_recommendations = 5
title = 'Love'
result = predict(title, df, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.007761955261230469 seconds to query forest.

 Top Recommendation(s) is(are) 
 544            Workin' Moms
438        2 Weeks in Lagos
247              Sweet Girl
569             Dreamy Eyes
25     Love on the Spectrum
Name: title, dtype: object


**Exhaustive Search**

In [16]:
!pip install faiss-cpu

In [17]:
import faiss

In [18]:
df['Index'] = df.index
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0,stop_words='english')
X_tfidf = tfidf.fit_transform(df['text'])
X_tfidf

<1000x34545 sparse matrix of type '<class 'numpy.float64'>'
	with 44837 stored elements in Compressed Sparse Row format>

In [19]:
vector = X_tfidf.toarray()
vector[0].shape[0]

34545

In [20]:
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [21]:
%%time
index = ExactIndex(vector, df["Index"])
index.build()

CPU times: user 85.8 ms, sys: 165 ms, total: 250 ms
Wall time: 257 ms


In [22]:
query_index = 5
print(f"The most similar items to item {df['Index'][query_index]} are:")

items_list = index.query(np.array([vector[query_index]]).astype('float32'))
df.loc[df['Index'].isin(items_list)]

The most similar items to item 5 are:


,title,director,release_year,listed_in,description,text,Index
5,Midnight Mass,Mike Flanagan,2021,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,Midnight Mass The arrival of a charismatic you...,5
86,Prey,Thomas Sieben,2021,"International Movies, Thrillers",A hiking trip into the wild turns into a despe...,Prey A hiking trip into the wild turns into a ...,86
307,Aftermath,Peter Winther,2021,Horror Movies,"Desperate to save their marriage, a young coup...","Aftermath Desperate to save their marriage, a ...",307
412,Chhota Bheem and the Incan Adventure,Rajiv Chilaka,2013,Children & Family Movies,"When Dholakpur’s princess is kidnapped, Bheem ...",Chhota Bheem and the Incan Adventure When Dhol...,412
643,The Seventh Day,Justin P. Lange,2021,Horror Movies,An inexperienced priest teams up with a harden...,The Seventh Day An inexperienced priest teams ...,643
745,Until Midnight,Tariq Alkazim,2019,"International Movies, Thrillers","After returning home from a work trip, a newly...",Until Midnight After returning home from a wor...,745
782,Bliss,Lance Young,1997,"Dramas, Romantic Movies",Two young newlyweds seek out the help of unlic...,Bliss Two young newlyweds seek out the help of...,782
866,Small Town Crime,"Eshom Nelms, Ian Nelms",2017,Thrillers,When a disgraced ex-cop discovers a dying woma...,Small Town Crime When a disgraced ex-cop disco...,866
918,Lava Ka Dhaava,NaN,2021,Reality TV,Actor Jaaved Jaafferi brings his signature hum...,Lava Ka Dhaava Actor Jaaved Jaafferi brings hi...,918
932,Green Zone,Paul Greengrass,2010,Action & Adventure,A US Army officer uncovers a conspiracy about ...,Green Zone A US Army officer uncovers a conspi...,932


**Product Quantization**

In [23]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
%%time
embeddings = model.encode(df['text'].tolist())
faiss.normalize_L2(embeddings)

CPU times: user 1min 11s, sys: 912 ms, total: 1min 12s
Wall time: 1min 11s


In [25]:
print("Shape of the Embeddings:",embeddings.shape)

Shape of the Embeddings: (1000, 768)


In [26]:
dim=768
ncentroids=50
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, 16 , 8)
index.train(embeddings) 
print(index.is_trained)
faiss.write_index(index, "trained.index")

True


In [27]:
ids = df['Index'].tolist()
ids = np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

1000


In [28]:
faiss.write_index(index,"block.index")

In [29]:
def searchFAISSIndex(data, id_col_name, query, index, nprobe, model, topk=20):
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)
  
    index.nprobe=nprobe
    
    D,I=index.search(query_embedding,topk) 
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [30]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

For a given query and it should give the top recommended titles matching the queries (in this case the top 20 and the cosine_sim and L2 scores.

In [35]:
query = "As her father nears the end of his life"
search_result = searchFAISSIndex(df, "Index", query, index, nprobe=10, model= model, topk=20)
search_result = search_result[['Index', 'title', 'description', 'cosine_sim', 'L2_score']]

In [36]:
search_result

,Index,title,description,cosine_sim,L2_score
3,160,Major Dad,When he marries a journalist and becomes stepd...,0.566016,0.931648
0,0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm...",0.555232,0.943152
6,438,2 Weeks in Lagos,A businessman returns home to Nigeria and fall...,0.548050,0.950737
15,795,Happy Endings,"After his fiancée, Alex, dumps him at the alta...",0.508905,0.991055
2,24,Jeans,When the father of the man she loves insists t...,0.499346,1.000654
9,498,Clash,When the patriarch of an emigrant Nigerian fam...,0.493881,1.006101
16,871,Sardar Ka Grandson,A devoted grandson’s mission to reunite his ai...,0.493855,1.006126
18,971,Between Worlds,"Grieving for his dead wife and daughter, a tru...",0.493037,1.006939
17,969,August: Osage County,"When their father disappears, three strong-wil...",0.486912,1.013004
12,662,Champions,"Years after getting his girlfriend pregnant, w...",0.480919,1.018902


**Trees and Graphs**

In [38]:
import nltk
from nltk.corpus import wordnet as wn
import annoy

Use AnnoyIndex to create a graph and run it on the tokenized vector created earlier using Tfidf

In [39]:
class TreeGraphIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention, metric='angular')
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [40]:
%%time
tg_index = TreeGraphIndex(vector, df['Index'])
tg_index.build()

CPU times: user 2.2 s, sys: 104 ms, total: 2.3 s
Wall time: 2.32 s


Query using an index and the results should be displayed

In [41]:
query_index = 5
print(f"The most similar items to item {df['Index'][query_index]} are:")
items_list = tg_index.query(vector[query_index])
df.loc[df['Index'].isin(items_list)]

The most similar items to item 5 are:


,title,director,release_year,listed_in,description,text,Index
5,Midnight Mass,Mike Flanagan,2021,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,Midnight Mass The arrival of a charismatic you...,5
86,Prey,Thomas Sieben,2021,"International Movies, Thrillers",A hiking trip into the wild turns into a despe...,Prey A hiking trip into the wild turns into a ...,86
307,Aftermath,Peter Winther,2021,Horror Movies,"Desperate to save their marriage, a young coup...","Aftermath Desperate to save their marriage, a ...",307
412,Chhota Bheem and the Incan Adventure,Rajiv Chilaka,2013,Children & Family Movies,"When Dholakpur’s princess is kidnapped, Bheem ...",Chhota Bheem and the Incan Adventure When Dhol...,412
643,The Seventh Day,Justin P. Lange,2021,Horror Movies,An inexperienced priest teams up with a harden...,The Seventh Day An inexperienced priest teams ...,643
745,Until Midnight,Tariq Alkazim,2019,"International Movies, Thrillers","After returning home from a work trip, a newly...",Until Midnight After returning home from a wor...,745
782,Bliss,Lance Young,1997,"Dramas, Romantic Movies",Two young newlyweds seek out the help of unlic...,Bliss Two young newlyweds seek out the help of...,782
866,Small Town Crime,"Eshom Nelms, Ian Nelms",2017,Thrillers,When a disgraced ex-cop discovers a dying woma...,Small Town Crime When a disgraced ex-cop disco...,866
918,Lava Ka Dhaava,NaN,2021,Reality TV,Actor Jaaved Jaafferi brings his signature hum...,Lava Ka Dhaava Actor Jaaved Jaafferi brings hi...,918
932,Green Zone,Paul Greengrass,2010,Action & Adventure,A US Army officer uncovers a conspiracy about ...,Green Zone A US Army officer uncovers a conspi...,932


**HSNW**

Create a new index using HSNW in Cosine similarity

In [42]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(vector)
index.createIndex({'post': 2}, print_progress=True)

In [43]:
ids, distances = index.knnQuery(vector[0], k=10)

Look for the nearest neighbours of the first data point

In [44]:
neighbours = index.knnQueryBatch(vector, k=10, num_threads=4)

In [45]:
neighbours

[(array([  0,  76, 971, 415, 604, 704, 827, 854, 606, 823], dtype=int32),
  array([3.5762787e-07, 9.6430832e-01, 9.6590835e-01, 9.7089767e-01,
         9.7264051e-01, 9.7450113e-01, 9.7733998e-01, 9.7824931e-01,
         9.7857738e-01, 9.7953868e-01], dtype=float32)),
 (array([  1, 684, 613, 108, 146, 584, 943, 173, 113, 869], dtype=int32),
  array([0.        , 0.95057994, 0.96222997, 0.969903  , 0.9718171 ,
         0.97365874, 0.9746312 , 0.97484994, 0.97529346, 0.9761698 ],
        dtype=float32)),
 (array([  2, 318, 346, 424, 310, 791, 711, 212,  42, 150], dtype=int32),
  array([0.        , 0.94609916, 0.9540503 , 0.95627904, 0.9584522 ,
         0.95858157, 0.9621711 , 0.973085  , 0.9734023 , 0.97543454],
        dtype=float32)),
 (array([  3, 616, 526, 957, 466, 184,  74, 691, 647, 856], dtype=int32),
  array([2.9802322e-07, 9.4733638e-01, 9.6415454e-01, 9.6443975e-01,
         9.6794474e-01, 9.6795738e-01, 9.6934712e-01, 9.6974313e-01,
         9.7006768e-01, 9.7683305e-01], dty